In [11]:
import numpy as np
import pandas as pd
'''
from collections import Counter
from astropy.io.fits import getdata
from astropy import units as u
from astropy.coordinates import SkyCoord, Angle
from astroquery.vizier import Vizier
from astropy.table import Table
from astroquery.xmatch import XMatch
from astroquery.simbad import Simbad
import time
'''


import warnings
warnings.filterwarnings('ignore')



In [12]:
field_name = 'CSCv2_TD_11162022'
TD = pd.read_csv(f'./data/{field_name}_MW_comb_cleaned.csv')
TD = TD.fillna('-99')
TD = TD.replace({-99:'-99'})

print(TD.loc[TD['GAIA_Source']!='-99','Class'].value_counts())

AGN        1020
YSO         590
LM-STAR     577
HM-STAR     115
CV           39
LMXB         25
HMXB         24
NS            7
Name: Class, dtype: int64


In [13]:
TD.loc[TD['GAIA_Source']!='-99', ['GAIA_Source','CSC__2CXO']].to_csv('./data/Gaia_11162022.csv',index=False)

In [14]:
TD.columns[:100]

#star_cols = ['GAIA_Source','TMASS__2MASS','ALLWISE_AllWISE','CATWISE_Name']

Index(['name', 'name_cat', 'ra_cat', 'dec_cat', 'Class', 'SubClass', 'ref',
       'GLAT', 'GLON', 'usrid', 'ra_pnt', 'dec_pnt', 'ra', 'dec',
       'err_ellipse_r0', 'err_ellipse_r1', 'err_ellipse_ang', 'significance',
       'extent_flag', 'pileup_flag', 'sat_src_flag', 'streak_src_flag',
       'conf_flag', 'flux_aper90_mean_b', 'e_flux_aper90_mean_b',
       'flux_aper90_mean_h', 'e_flux_aper90_mean_h', 'flux_aper90_mean_m',
       'e_flux_aper90_mean_m', 'flux_aper90_mean_s', 'e_flux_aper90_mean_s',
       'kp_intra_prob_b', 'ks_intra_prob_b', 'var_inter_prob_b', 'nh_gal',
       'flux_powlaw_mean', 'e_flux_powlaw_mean', 'powlaw_gamma_mean',
       'e_powlaw_gamma_mean', 'powlaw_nh_mean', 'e_powlaw_nh_mean',
       'powlaw_ampl_mean', 'e_powlaw_ampl_mean', 'powlaw_stat', 'remove_code',
       'flux_significance_b_max', 'flux_aper90_ave_s', 'e_flux_aper90_ave_s',
       'flux_aper90_ave_m', 'e_flux_aper90_ave_m', 'flux_aper90_ave_h',
       'e_flux_aper90_ave_h', 'chisqr', 'dof', '

In [15]:
TD['Class'].value_counts()
print(len(TD))

TD_remove = TD[(TD['GAIA_Source']=='-99') & (TD['TMASS__2MASS']=='-99') & (TD['ALLWISE_AllWISE']=='-99') & (TD['CATWISE_Name']=='-99') & (TD['Class'].isin(['LM-STAR','HM-STAR','YSO']))]
print(TD['Class'].value_counts())
print(TD_remove['Class'].value_counts())
TD = TD[~TD['name'].isin(TD_remove['name'])].reset_index(drop=True)
print(TD['Class'].value_counts())


3126
AGN        1405
YSO         748
LM-STAR     607
HM-STAR     128
NS           95
LMXB         63
CV           47
HMXB         33
Name: Class, dtype: int64
YSO        14
LM-STAR     1
Name: Class, dtype: int64
AGN        1405
YSO         734
LM-STAR     606
HM-STAR     128
NS           95
LMXB         63
CV           47
HMXB         33
Name: Class, dtype: int64


In [16]:
# combine ALLWISE, CatWISE and UnWISE

for w in ['W1','W2']:
    TD.loc[TD['ALLWISE_AllWISE']=='-99', w+'mag']      = TD.loc[TD['ALLWISE_AllWISE']=='-99', w+'mproPM']
    TD.loc[TD['ALLWISE_AllWISE']=='-99', 'e_'+w+'mag'] = TD.loc[TD['ALLWISE_AllWISE']=='-99', 'e_'+w+'mproPM']




In [17]:
TD[['Class','GAIA_Source','TMASS__2MASS','ALLWISE_AllWISE','CATWISE_Name']]

,Class,GAIA_Source,TMASS__2MASS,ALLWISE_AllWISE,CATWISE_Name
0,LM-STAR,2848205298895660544.0,00015054+2330153,J000150.53+233015.5,J000150.52+233015.5
1,NS,-99,-99,-99,-99
2,LM-STAR,2860586826320959488.0,00182643+3004000,J001826.43+300400.0,J001826.42+300400.1
3,LM-STAR,423793597149050816.0,00523995+5628372,J005239.96+562837.1,J005239.97+562837.1
4,LM-STAR,313508874324686208.0,01070567+3210569,J010705.70+321056.8,J010705.71+321056.7
...,...,...,...,...,...
3106,AGN,2738473415672342016.0,-99,J235718.38+004350.2,J235718.38+004350.3
3107,AGN,2738267669559100416.0,23583317+0034535,J235833.21+003453.7,J235833.21+003453.6
3108,AGN,6488149125448220672.0,23590432-6054590,J235904.30-605459.1,J235904.30-605459.1
3109,AGN,2335101047989043200.0,-99,J235939.78-250056.6,J235939.79-250056.7


In [18]:
TD.columns[100:]

Index(['ALLWISE_RA', 'ALLWISE_DEC', 'ALLWISE_err0', 'ALLWISE_err1',
       'ALLWISE_errPA', 'ALLWISE_AllWISE', 'ALLWISE__r', 'W1mag', 'W2mag',
       'W3mag', 'W4mag', 'e_W1mag', 'e_W2mag', 'e_W3mag', 'e_W4mag',
       'CATWISE_RA', 'CATWISE_DEC', 'CATWISE_PU', 'CATWISE_Name', 'CATWISE__r',
       'W1mproPM', 'W2mproPM', 'e_W1mproPM', 'e_W2mproPM',
       'Separation_GAIA_CSC', 'Separation_GAIA_CSC_ra',
       'Separation_GAIA_CSC_dec', 'Separation_TMASS_CSC',
       'Separation_TMASS_CSC_ra', 'Separation_TMASS_CSC_dec',
       'Separation_TMASS_GAIA', 'Separation_TMASS_GAIA_ra',
       'Separation_TMASS_GAIA_dec', 'Separation_ALLWISE_CSC',
       'Separation_ALLWISE_CSC_ra', 'Separation_ALLWISE_CSC_dec',
       'Separation_ALLWISE_GAIA', 'Separation_ALLWISE_GAIA_ra',
       'Separation_ALLWISE_GAIA_dec', 'Separation_ALLWISE_TMASS',
       'Separation_ALLWISE_TMASS_ra', 'Separation_ALLWISE_TMASS_dec',
       'Separation_CATWISE_CSC', 'Separation_CATWISE_CSC_ra',
       'Separation_CATW

In [19]:
TD = TD.rename(columns = {
    'flux_aper90_ave_s':'Fcsc_s','e_flux_aper90_ave_s':'e_Fcsc_s','flux_aper90_ave_m':'Fcsc_m','e_flux_aper90_ave_m':'e_Fcsc_m',
    'flux_aper90_ave_h':'Fcsc_h','e_flux_aper90_ave_h':'e_Fcsc_h','kp_prob_b_max':'var_intra_prob'})




In [20]:
TD.columns[:100]
TD = TD.replace({'-99':np.nan})

TD.to_csv(f'./data/{field_name}_prepare_cols.csv',index=False)

In [10]:
TD_old = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_CSCv2/files/CSC_TD_MW_remove.csv')

In [11]:
TD_old.columns[:100]



Index(['name', 'usrid', 'ra', 'dec', 'err_ellipse_r0', 'err_ellipse_r1',
       'err_ellipse_ang', 'significance', 'extent_flag', 'pileup_flag',
       'sat_src_flag', 'streak_src_flag', 'conf_flag', 'flux_aper90_mean_b',
       'e_flux_aper90_mean_b', 'flux_aper90_mean_h', 'e_flux_aper90_mean_h',
       'flux_aper90_mean_m', 'e_flux_aper90_mean_m', 'flux_aper90_mean_s',
       'e_flux_aper90_mean_s', 'kp_intra_prob_b', 'ks_intra_prob_b',
       'var_inter_prob_b', 'nh_gal', 'flux_powlaw_mean', 'e_flux_powlaw_mean',
       'powlaw_gamma_mean', 'e_powlaw_gamma_mean', 'powlaw_nh_mean',
       'e_powlaw_nh_mean', 'powlaw_ampl_mean', 'e_powlaw_ampl_mean',
       'powlaw_stat', 'remove_code', 'flux_significance_b_max',
       'flux_aper90_ave_s', 'e_flux_aper90_ave_s', 'flux_aper90_ave_m',
       'e_flux_aper90_ave_m', 'flux_aper90_ave_h', 'e_flux_aper90_ave_h',
       'chisqr', 'dof', 'kp_prob_b_max', 'var_inter_prob', 'significance_max',
       'flux_flag', 'flux_aper90_ave_b', 'e_flux_ap

In [12]:
TD_old

,name,usrid,ra,dec,err_ellipse_r0,err_ellipse_r1,err_ellipse_ang,significance,extent_flag,pileup_flag,...,PU_e_plx_allwise,PU_epsi_allwise,X_PU_allwise,cp_flag_allwise,_r2_allwise,W1mag_unwise,W2mag_unwise,e_W1mag_unwise,e_W2mag_unwise,CSC_flags
0,2CXO J000009.3+135618,1,0.039125,13.938494,0.79,0.77,64.2,1.95,False,0.0,...,0.000409,0.000842,0.818734,0,NaN,15.583786,14.656088,-0.014574,-0.024704,NaN
1,2CXO J000230.7+004959,2,0.627958,0.833072,0.72,0.72,64.9,11.07,False,0.0,...,0.000327,0.000000,0.729833,0,NaN,14.450252,13.126989,-0.006823,-0.008069,NaN
2,2CXO J000622.6-000424,4,1.594333,-0.073572,0.78,0.76,40.5,25.42,True,0.0,...,0.001305,0.001682,0.802858,0,NaN,15.290073,14.280817,-0.012854,-0.019867,extent
3,2CXO J000659.2-001740,5,1.747042,-0.294661,0.84,0.79,109.8,4.11,False,0.0,...,0.000716,0.001624,0.858203,-8,NaN,16.488542,15.621306,-0.035409,-0.065102,NaN
4,2CXO J000703.6+155423,6,1.765000,15.906575,0.72,0.72,25.4,8.63,False,0.0,...,0.000450,0.001256,0.723038,0,NaN,11.722830,10.673928,-0.001297,-0.001676,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2957,2CXO J183000.6+011340,3165,277.502750,1.227794,0.71,0.71,60.6,20.82,False,0.0,...,0.001463,0.006324,0.714386,0,NaN,9.767272,9.394435,-0.000525,-0.000957,NaN
2958,2CXO J183001.1+011324,3166,277.504625,1.223367,0.75,0.74,31.6,8.10,False,0.0,...,NaN,NaN,0.752675,0,NaN,11.242325,9.863729,-0.001150,-0.001209,NaN
2959,2CXO J183003.4+011619,3167,277.514208,1.271981,0.73,0.72,59.9,13.69,False,0.0,...,0.000106,0.000278,0.732346,0,NaN,9.734526,9.157654,-0.000506,-0.000784,NaN
2960,2CXO J183007.7+011204,3168,277.532167,1.201211,0.75,0.74,33.6,15.60,True,0.0,...,0.000528,0.003738,0.752885,0,NaN,9.227420,8.482405,-0.000386,-0.000555,extent


In [13]:
TD

,name,name_cat,ra_cat,dec_cat,Class,SubClass,ref,GLAT,GLON,usrid,...,B_rgeo,rpgeo,b_rpgeo,B_rpgeo,gaiadist_flag,_q,W1mag_comb,e_W1mag_comb,W2mag_comb,e_W2mag_comb
0,2CXO J000150.5+233015,G17197224783680,0.460598,23.504296,LM-STAR,K3,LAMOST-DR8-AFGK,108.467901,-37.970470,1,...,190.873550,190.309402,189.739410,190.800888,10033.0,1,10.510,0.021,10.582,0.019
1,2CXO J000701.5+730308,J0007+7303,1.757083,73.052056,NS,NRAD,2005AJ....129.1993M,119.659483,10.463306,2,...,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN
2,2CXO J001826.4+300400,GSC 2261-1462,4.610125,30.066639,LM-STAR,G5,1971PASP...83..810T,114.487557,-32.263400,3,...,1109.249020,1081.566890,1064.833860,1099.735470,10033.0,3,12.206,0.023,12.275,0.025
3,2CXO J005240.0+562837,G16901528933076,13.166510,56.477011,LM-STAR,G8,LAMOST-DR8-AFGK,123.102718,-6.394337,4,...,281.259705,280.002930,278.943024,281.291962,10033.0,4,10.354,0.023,10.384,0.022
4,2CXO J010705.6+321056,G17222139826623,16.773656,32.182487,LM-STAR,K4,LAMOST-DR8-AFGK,126.779542,-30.572263,6,...,348.501251,337.011963,327.051025,349.826843,10033.0,6,11.952,0.024,11.922,0.023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,2CXO J235718.3+004350,SDSS J23573+0043,359.326667,0.730556,AGN,Q;*|QSO|X,2010A&A...518A..10V;SIMBAD,95.698536,-59.248481,3577,...,3259.827640,2939.993410,2385.917240,3454.502690,10033.0,3577,16.198,0.070,16.105,0.213
3137,2CXO J235833.2+003453,SDSS J23585+0034,359.638333,0.581389,AGN,A;*|AGN|G|Sy1,2010A&A...518A..10V;SIMBAD,96.140239,-59.511001,3578,...,NaN,NaN,NaN,NaN,NaN,3578,14.292,0.030,13.952,0.040
3138,2CXO J235904.2-605459,PKS 2356-61,359.767917,-60.916389,AGN,A;Sy2;*|AGN|G|GiC|Rad|Sy2|X|cm|rG,2010A&A...518A..10V;2018ApJS..235....4O;SIMBAD,314.018635,-55.070324,3579,...,NaN,NaN,NaN,NaN,NaN,3579,12.631,0.023,11.872,0.020
3139,2CXO J235939.7-250056,XMM J23596-2500,359.915833,-25.015833,AGN,Q;*|AGN|QSO|Sy1|X,2010A&A...518A..10V;SIMBAD,40.331786,-78.188942,3580,...,3020.735600,5175.990230,4410.054690,6247.561040,10033.0,3580,15.243,0.038,14.127,0.043


In [34]:
TD_old.loc[(~TD_old['name'].isin(TD['name'])) & ((TD_old['Class']=='HMXB')),'name'].values


array(['2CXO J143308.3-611540', '2CXO J170025.2-421900',
       '2CXO J175834.5-212321'], dtype=object)

In [35]:
print(TD_old['Class'].value_counts())
print(TD['Class'].value_counts())


AGN        1390
YSO        1024
LM-STAR     208
HM-STAR     118
NS           87
CV           44
LMXB         41
HMXB         26
NS_BIN       24
Name: Class, dtype: int64
AGN        1405
YSO         734
LM-STAR     636
HM-STAR     129
NS           95
LMXB         63
CV           47
HMXB         32
Name: Class, dtype: int64


In [36]:
from TD_dictionary import Simbad_dict,crowd_fields_dict,rare_sources_removed_dict,rare_sources_saving_dict 

src_missing = ['2CXO J143308.3-611540', '2CXO J170025.2-421900',
       '2CXO J175834.5-212321']

In [37]:
for src in src_missing:
    if src in rare_sources_removed_dict:
        print(src, rare_sources_removed_dict[src])
        
for src in src_missing:
    if src in rare_sources_saving_dict:
        print(src, rare_sources_saving_dict[src])
        
src_missing2 = ['2CXO J143308.3-611540','2CXO J175834.5-212321']

2CXO J170025.2-421900 AX J1700.2-4220, ambiguious source


In [23]:
TD_simbad = pd.read_csv(f'./data/{field_name}_SIMBAD.csv')
TD_simbad['name']

0        2CXO J000000.0+004331
1        2CXO J000001.8-094653
2        2CXO J000002.2+003739
3        2CXO J000002.9-350332
4        2CXO J000004.7+004654
                 ...          
93883    2CXO J235953.6-124148
93884    2CXO J235957.0+135643
93885    2CXO J235957.5+003919
93886    2CXO J235958.2+005140
93887    2CXO J235958.8+004206
Name: name, Length: 93888, dtype: object

In [24]:
TD_simbad.loc[TD_simbad['name'].isin(src_missing2), ['Class','ref','remove_regions','name','ra','dec']]

,Class,ref,remove_regions,name,ra,dec
61454,HMXB,2018ApJS..235....4O,NaN,2CXO J143308.3-611540,218.284719,-61.261119
61455,HMXB,INTEGRAL General Reference Catalog,NaN,2CXO J143308.3-611540,218.284719,-61.261119
61456,Star,SIMBAD,NaN,2CXO J143308.3-611540,218.284719,-61.261119
78687,HMXB,2018ApJS..235....4O,|GalacticCenter,2CXO J175834.5-212321,269.644027,-21.389373
78688,HMXB,INTEGRAL General Reference Catalog,|GalacticCenter,2CXO J175834.5-212321,269.644027,-21.389373
78689,HMXB,SIMBAD,|GalacticCenter,2CXO J175834.5-212321,269.644027,-21.389373
